In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV,cross_val_score
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

In [2]:
selected_features=['partner',
 'onlinebackup',
 'tenure',
 'contract',
 'monthlycharges',
 'totalcharges',
 'techsupport', 'internetservice',
 'multiplelines']


In [3]:
x_train_bal=pd.read_csv("../datasets/x_train_bal.csv")
y_train_bal=pd.read_csv("../datasets/y_train_bal.csv")
x_train_bal=x_train_bal[selected_features]

In [4]:
x_train_bal.head()

,partner,onlinebackup,tenure,contract,monthlycharges,totalcharges,techsupport,internetservice,multiplelines
0,1,1,65,2,94.55,6078.75,1,2,1
1,0,0,26,0,35.75,1022.50,1,1,2
2,1,1,68,2,90.20,6297.65,1,2,1
3,0,1,3,0,84.30,235.05,0,2,0
4,1,0,49,0,40.65,2070.75,0,1,2


In [19]:
y_train_bal.value_counts()

churn
0        4130
1        4130
Name: count, dtype: int64

Using GridSearch to find best parameters.

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy'],
    'class_weight': ['balanced', None],
    'max_leaf_nodes': [None, 20, 50],
    'min_impurity_decrease': [0.0, 0.01]
}

In [8]:
grid_skf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=3,               # <-- use StratifiedKFold here
    scoring='accuracy',
    verbose=3,
    n_jobs=-1 
)

In [9]:
grid_skf.fit(x_train_bal, y_train_bal)
print("Best params (StratifiedKFold):", grid_skf.best_params_)

Fitting 3 folds for each of 10368 candidates, totalling 31104 fits


KeyboardInterrupt: 

 {'bootstrap': True, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100, 'verbose': 0, 'warm_start': False}

In [ ]:
grid_skf.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
grid_skf2 = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,               # <-- use StratifiedKFold here
    scoring='accuracy',
    verbose=3,
    n_jobs=-1 
)
grid_skf2.fit(x_train_bal, y_train_bal)
print("Best params (StratifiedKFold):", grid_skf.best_params_)

Fitting 5 folds for each of 1728 candidates, totalling 8640 fits


c:\Users\PUNEETH\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best params (StratifiedKFold): {'bootstrap': True, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 20, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200, 'verbose': 0, 'warm_start': False}


In [ ]:
grid_skf2.best_score_

np.float64(0.82409200968523)

In [ ]:
grid_skf2.best_params_

{'bootstrap': True,
 'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': 20,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 200,
 'verbose': 0,
 'warm_start': False}

In [20]:
rf_model = RandomForestClassifier(
    bootstrap=True,
    class_weight='balanced',
    criterion='gini',
    max_depth=20,
    max_features='sqrt',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_samples_leaf=2,
    min_samples_split=2,
    n_estimators=200,
    verbose=0,
    warm_start=False
)

In [21]:
rf_model.fit(x_train_bal, y_train_bal)

c:\Users\PUNEETH\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(class_weight='balanced', max_depth=20,
                       min_samples_leaf=2, n_estimators=200)

In [22]:
x_test=pd.read_csv("../datasets/x_test.csv")
y_test=pd.read_csv("../datasets/y_test.csv")
x_test=x_test[selected_features]

In [23]:
y_pred_rf = rf_model.predict(x_test)
y_prob_rf = rf_model.predict_proba(x_test)[:, 1]

In [24]:
test_accuracy = accuracy_score(y_test, y_pred_rf)
test_auc = roc_auc_score(y_test, y_prob_rf)

print(f"\nTest Accuracy: {round(test_accuracy, 4)}")
print(f"Test AUC: {round(test_auc, 4)}")


Test Accuracy: 0.7548
Test AUC: 0.8041


In [15]:
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_distributions=param_grid,
    n_iter=200,                # number of parameter settings sampled
    cv=5,                     # 5-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1                 # use all processors
)
random_search.fit(x_train_bal, y_train_bal)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


c:\Users\PUNEETH\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=200, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'class_weight': ['balanced', None],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': [10, 20, 30, None],
                                        'max_features': ['sqrt', 'log2'],
                                        'max_leaf_nodes': [None, 20, 50],
                                        'min_impurity_decrease': [0.0, 0.01],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300]},
                   random_state=42, verbose=2)

In [ ]:
random_search.best_score_

np.float64(0.8221549636803873)

In [ ]:
random_search.best_params_

{'n_estimators': 300,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'min_impurity_decrease': 0.0,
 'max_leaf_nodes': None,
 'max_features': 'log2',
 'max_depth': 20,
 'criterion': 'gini',
 'class_weight': 'balanced',
 'bootstrap': True}

Since GridSearchCV was taking very long time execute.
Using RandomizedSearchCV for model tunning

In [ ]:
random_search2 = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_distributions=param_grid,
    n_iter=2000,                # number of parameter settings sampled
    cv=5,                     # 5-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1                 # use all processors
)
random_search2.fit(x_train_bal, y_train_bal)

Fitting 5 folds for each of 2000 candidates, totalling 10000 fits


c:\Users\PUNEETH\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=2000, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'class_weight': ['balanced', None],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': [10, 20, 30, None],
                                        'max_features': ['sqrt', 'log2'],
                                        'max_leaf_nodes': [None, 20, 50],
                                        'min_impurity_decrease': [0.0, 0.01],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300]},
                   random_state=42, verbose=2)

In [ ]:
random_search2.best_score_

np.float64(0.82409200968523)

In [ ]:
random_search2.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'min_impurity_decrease': 0.0,
 'max_leaf_nodes': None,
 'max_features': 'log2',
 'max_depth': 20,
 'criterion': 'gini',
 'class_weight': None,
 'bootstrap': True}

In [12]:
selected_parameters={
 'n_estimators': 200,        
 'min_samples_split': 2,              
 'min_samples_leaf': 2,               
 'min_impurity_decrease': 0.0,        
 'max_leaf_nodes': None,               
 'max_features': 'log2',              
 'max_depth': 20,        
 'criterion': 'gini',                 
 'class_weight': None,                
 'bootstrap': True                     
}


In [13]:
final_model=RandomForestClassifier(**selected_parameters)

In [14]:
final_fit=final_model.fit(x_train_bal,y_train_bal)

c:\Users\PUNEETH\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [15]:
x_test=pd.read_csv("../datasets/x_test.csv")
y_test=pd.read_csv("../datasets/y_test.csv")
x_test=x_test[selected_features]

In [16]:
y_predict=final_fit.predict(x_test)

In [25]:
y_proba = final_fit.predict_proba(x_test)[:, 1]  # probability of class 1

# Accuracy
test_accuracy = accuracy_score(y_test, y_predict)

# ROC AUC (using probabilities)
test_accuracy = accuracy_score(y_test, y_predict)
test_auc = roc_auc_score(y_test, y_proba)

print(f"\nTest Accuracy: {round(test_accuracy, 4)}")
print(f"Test AUC: {round(test_auc, 4)}")


Test Accuracy: 0.7619
Test AUC: 0.804


To save the model

In [26]:
import joblib
joblib.dump(final_fit, '../churn_predicter.pkl')

['../churn_predicter.pkl']